# Setup Code
**DO NOT CHANGE THIS!!**
**RUN THE CELL BELOW FIRST!!**

In [ ]:
import carla
import os

client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
if client_name == "DOES NOT EXIST":
    raise Exception("The environment variable for the container name of the carla server has not been set")

# Connect to the client and retrieve the world object
client = carla.Client(client_name, 2000)
world = client.get_world()
print("Got world")

In [ ]:
print(world)
print(worl

# Start Adding Code or Markdown Cells Below
Some helpful code cells are provided below that you may choose to use

In [ ]:
#### SPAWN VEHICLES ####
"""
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points)))
"""
#### SET ALL VEHICLES TO AUTOPILOT
"""
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)
"""

In [ ]:
!pip install pillow

In [ ]:
import carla
import os
from PIL import Image
from io import BytesIO
from IPython.display import display
from IPython.display import HTML
from IPython.display import Image as IPythonImage, display, clear_output
import random
import time
import base64 

# Use this to set a fixed size for images in the notebook display
IMAGE_WIDTH = 640

# Assuming client and world are already set up as per your provided code

# Get the world and blueprint library
world = client.get_world()
blueprint_library = world.get_blueprint_library()

# Find the Tesla Model 3 blueprint
tesla_model_3_bp = blueprint_library.find('vehicle.tesla.model3')

# Choose a spawn point
spawn_point = random.choice(world.get_map().get_spawn_points())

# Spawn the Tesla
tesla_model_3 = world.spawn_actor(tesla_model_3_bp, spawn_point)

# Enable autopilot
tesla_model_3.set_autopilot(True)

# Add a camera sensor to the Tesla
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=tesla_model_3)

# Add a LiDAR sensor to the Tesla
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_transform = carla.Transform(carla.Location(x=0, z=2.5))
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=tesla_model_3)

# Function to convert CARLA image to a base64 string
def img_to_base64_str(image):
    image_pil = carla_image_to_pil(image)
    buffered = BytesIO()
    image_pil.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("ascii")


# Function to convert CARLA image to PIL Image
def carla_image_to_pil(image):
    image.convert(carla.ColorConverter.Raw)
    image_bytes = bytes(image.raw_data)
    image_pil = Image.frombytes("RGBA", (image.width, image.height), image_bytes, "raw", "BGRA")
    return image_pil.convert("RGB")  # Convert to RGB


# Function to process camera images
def process_image(image):
    image_pil = carla_image_to_pil(imge)
    # Display the image
    display(image_pil)

# Listen to the camera's image capture event
camera.listen(lambda image: process_image(image))
